**Backoff-model:**

Katz back-off is a generative n-gram language model that estimates the conditional probability of a word given its history in the n-gram. It accomplishes this estimation by backing off through progressively shorter history models under certain conditions. By doing so, the model with the most reliable information about a given history is used to provide the better results.

Pros:
- Reduction in model size.
- Better accuracy as compared to our baseline model.

Cons:
- Sensitive to noise
- It can’t describe the relation between words longer
than N.
- Success probability degrades with increasing number of nodes.

In [5]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
import os
print(os.listdir('../gdrive/My Drive/NLP/Project/Final Project/LSTM'))
os.chdir('../gdrive/My Drive/NLP/Project/Final Project/LSTM')

In [0]:
# Python library imports:
import re
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.tokenize import word_tokenize, WhitespaceTokenizer, TweetTokenizer
np.random.seed(seed=234)

In [0]:
# The below reads in N lines of text from the 40-million-word news corpus I used (provided by SwiftKey for educational purposes).
N = 15000
with open("republic.txt") as myfile:
    articles = [next(myfile) for x in range(N)]
joined_articles = [" ".join(articles)]
# The below takes out anything that's not a letter, replacing it with a space, as well as any single letter that is not the pronoun "I" or the article "a."
def clean_article(article):
    art1 = re.sub("[^A-Za-z]", ' ', article)
    art2 = re.sub("\s[B-HJ-Zb-hj-z]\s", ' ', art1)
    art3 = re.sub("^[B-HJ-Zb-hj-z]\s", ' ', art2)
    art4 = re.sub("\s[B-HJ-Zb-hj-z]$", ' ', art3)
    return art4.lower()

In [0]:
# The below breaks up the words into n-grams of length 1 to 5 and puts their counts into a Pandas dataframe with the n-grams as column names.  The maximum number of n-grams can be specified if a large corpus is being used.
ngram_bow = CountVectorizer(stop_words = None, preprocessor = clean_article, tokenizer = WhitespaceTokenizer().tokenize, ngram_range=(1,5), max_features = None, max_df = 1.0, min_df = 1, binary = False)
ngram_count_sparse = ngram_bow.fit_transform(joined_articles)
ngram_count = pd.DataFrame(ngram_count_sparse.toarray())
ngram_count.columns = ngram_bow.get_feature_names()

In [0]:
# The below turns the n-gram-count dataframe into a Pandas series with the n-grams as indices for ease of working with the counts.  The second line can be used to limit the n-grams used to those with a count over a cutoff value.
sums = ngram_count.sum(axis = 0)
sums = sums[sums > 0]
ngrams1 = list(sums.index.values)
# The function below gives the total number of occurrences of 1-grams in order to calculate 1-gram frequencies
def number_of_onegrams(sums):
    onegrams = 0
    for ng in ngrams1:
        ng_split = ng.split(" ")
        if len(ng_split) == 1:
            onegrams += sums[ng]
    return onegrams

In [0]:
# The function below makes a series of 1-gram frequencies.  This is the last resort of the back-off algorithm if the n-gram completion does not occur in the corpus with any of the prefix words.
def base_freq(og):
    freqs = pd.Series()
    for ng in ngrams1:
        ng_split = ng.split(" ")
        if len(ng_split) == 1:
            freqs[ng] = sums[ng] / og
    return freqs
# For use in later functions so as not to re-calculate multiple times:
bf = base_freq(number_of_onegrams(sums))

In [0]:
# The function below finds any n-grams that are completions of a given prefix phrase with a specified number (could be zero) of words 'chopped' off the beginning.  For each, it calculates the count ratio of the completion to the (chopped) prefix, tabulating them in a series to be returned by the function.  If the number of chops equals the number of words in the prefix (i.e. all prefix words are chopped), the 1-gram base frequencies are returned.
def find_completion_scores(prefix, chops, factor = 0.4):
    cs = pd.Series()
    prefix_split = prefix.split(" ")
    l = len(prefix_split)
    prefix_split_chopped = prefix_split[chops:l]
    new_l = l - chops
    if new_l == 0:
        return factor**chops * bf
    prefix_chopped = ' '.join(prefix_split_chopped)
    for ng in ngrams1:
        ng_split = ng.split(" ")
        if (len(ng_split) == new_l + 1) and (ng_split[0:new_l] == prefix_split_chopped):
            cs[ng_split[-1]] = factor**chops * sums[ng] / sums[prefix_chopped]
    return cs

In [56]:
# Example of completion scores:
find_completion_scores('the rest of', 0, 0.4)

greece     0.043478
mankind    0.086957
our        0.086957
the        0.739130
their      0.043478
dtype: float64

In [0]:
# The below tries different numbers of 'chops' up to the length of the prefix to come up with a (still unordered) combined list of scores for potential completions of the prefix.
def score_given(given, fact = 0.4):
    sg = pd.Series()
    given_split = given.split(" ")
    given_length = len(given_split)
    for i in range(given_length+1):
        fcs = find_completion_scores(given, i, fact)
        for i in fcs.index:
            if i not in sg.index:
                sg[i] = fcs[i]
    return sg
#The below takes the potential completion scores, puts them in descending order and re-normalizes them as a percentage (pseudo-probability).
def score_output(given, fact = 0.4):
    sg = score_given(given, fact)
    ss = sg.sum()
    sg = 100 * sg / ss
    sg.sort_values(axis=0, ascending=False, inplace=True)
    return round(sg,1)

In [58]:
# The below shows that even with a corpus that is way too small, results start to become intuitive.
score_output('the rest of', fact = 0.4)[0:15]

the          64.5
mankind       7.6
our           7.6
greece        3.8
their         3.8
a             0.6
them          0.3
his           0.3
of            0.3
all           0.2
good          0.2
plato         0.2
which         0.2
justice       0.1
knowledge     0.1
dtype: float64

In [0]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from keras.utils import plot_model
import nltk
from nltk.util import ngrams
import collections

In [61]:
# code to confirm the n-grams generated by the our model by comparing it with actual frequency based output
blogs = open('republic.txt', 'rt')
text = blogs.read()
blogs.close()
corpus = text
# We will remove double quotes, !, ? $, #, etc
# specify to translate chars 
str1 = ""
# specify to replace with 
str2 = ""
# delete chars 
str3 = "\"!?#$%^&*+"

trg = corpus
table = trg.maketrans(str1, str2, str3)
corpus = trg.translate(table)
words = corpus.split()

esQuadgrams = ngrams(words, 4)
esQuadgramFreq = collections.Counter(esQuadgrams)
esQuadgramFreq.most_common(10)

[(('for', 'the', 'sake', 'of'), 28),
 (('Very', 'true,', 'he', 'said.'), 26),
 (('the', 'interest', 'of', 'the'), 25),
 (('What', 'do', 'you', 'mean'), 24),
 (('at', 'the', 'same', 'time'), 22),
 (('the', 'rest', 'of', 'the'), 21),
 (('Yes,', 'he', 'said,', 'that'), 21),
 (('in', 'the', 'case', 'of'), 20),
 (('as', 'well', 'as', 'of'), 18),
 (('the', 'idea', 'of', 'good'), 18)]

In [63]:
# code to test the LSTM generated output with the frequency-based output over 100 predictions.
testSet = esQuadgramFreq.most_common(100)
count=0
for quadgramTup in testSet:
  quadgram=quadgramTup[0]
  seed_text=quadgram[0]+" "+quadgram[1]+" "+quadgram[2]
  expected_text=quadgram[3]
  generated = score_output(seed_text, fact = 0.4)[0:1].index[0]
  if generated.strip()==expected_text:
    count+=1
print ("Accuracy: ",count/100)

Accuracy:  0.7
